In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML
import pyperclip
import pandas as pd
from misc_util.pretty_print import Markdown, display

from misc_util.logutils import setup_logging
from misc_util.pretty_print import Markdown, display

from derive_conceptualspace.pipeline import SnakeContext, load_envfiles
from derive_conceptualspace.util.result_analysis_tools import getfiles_allconfigs, display_metrics, show_lambda_elements, highlight_nonzero_max
from derive_conceptualspace.settings import DEFAULT_N_CPUS
from derive_conceptualspace.util.threadworker import WorkerPool
from derive_conceptualspace.cli.args_from_filename import get_filename, print_envvars

plt.rcParams['figure.figsize'] = [16, 10]

from derive_conceptualspace.evaluate.shallow_trees import classify_shallowtree, classify_shallowtree_multi
from derive_conceptualspace.util.desc_object import DescriptionList
from derive_conceptualspace.pipeline import featureaxes_loader, cluster_loader

## Prepare all parameter-combinations

In [ ]:
setup_logging()
load_envfiles("placetypes")
configs, print_cnf = getfiles_allconfigs("clusters", parse_all=True, verbose=True)

In [ ]:
print_envvars(get_filename(configs[0], get_dependencies=False, doprint=False))

In [ ]:
ctx = SnakeContext.loader_context(config=configs[0], silent=False)
descriptions, clusters, embedding = ctx.load("pp_descriptions", "clusters", "embedding", 
                                             loaders=dict(pp_descriptions=DescriptionList.from_json, clusters=cluster_loader, embedding=lambda **args: args["embedding"].embedding_))

In [ ]:
res = classify_shallowtree_multi(clusters, embedding, descriptions, verbose=True)

In [ ]:
SHOW_CROSSVAL_NUM = 5

styles = [{'selector': 'th', 'props': [('vertical-align','top'),('text-align','left')]}] #('border-style', 'solid')  #see https://stackoverflow.com/a/55904239/5122790
styler = lambda df: df.style.apply(highlight_nonzero_max, axis=0).format(precision=3).set_table_styles(styles)
    
for taxonomy in set(res.index.get_level_values(0)):
    display(Markdown(f"## {taxonomy}"))
    df = res.xs(SHOW_CROSSVAL_NUM, level=1).xs(taxonomy).unstack(level=[1])
    display(styler(df))